In [1]:
import os
import numpy
import pandas
from functools import partial, reduce

# References

1. [Investopedia](http://i.investopedia.com/inv/pdf/tutorials/dcf.pdf)
2. [...](https://.com)

## Equation

$$ DCF = \frac{CF_1}{(1 + r) ^ 1} + \frac{CF_2}{(1 + r) ^ 2} + ... + \frac{CF_n}{(1 + r) ^ n} $$

Where CF stands for unlevered free cash flow for a given year ($CF_1$ means cash flow in year 1, and so on); and $r$ stands for the discount rate.

CF = EBITDA - capital expenditure - changes in net working capital - taxes

## Load Data

In [2]:
sample_path = os.getcwd() + '/sample-output/'

In [3]:
def get_dataframe(year):
    
    file_name = [x for x in os.listdir('sample-output/') if x.endswith('{}.csv'.format(year))]
    
    dfs = []
    for f in file_name:
        dfs.append(pandas.read_csv(sample_path + f))
        
    return reduce(partial(pandas.merge, on = 'ticker_code'), dfs)

In [4]:
dfs = []
for year in (2017, 2018):
    dfs.append(get_dataframe(year))

In [5]:
df = pandas.concat(dfs, sort = False).reset_index(drop = True)

## Calculate CF

In [6]:
# cf = pretax_income - fixed_assets - (net working capital in T1 - net working capital in T0) - tax_expenses
# net working capital = total_current_assets - total_current_liabilities

In [7]:
def get_free_cash_flow(year):
    
    # select necessary variables
    # sort by ticker code in ascending order
    # reset index
    df_nwc = df[['year', 'ticker_code', 'total_current_assets', 'total_current_liabilities']]\
    .sort_values(by=['ticker_code'], ascending=True)\
    .reset_index(drop=True)
    
    # add net working capital as a column
    df_nwc['net_working_capital'] = df_nwc['total_current_assets'] - df_nwc['total_current_liabilities']
    
    # changes in net working capital
    df_nwc['net_working_capital_delta'] = df_nwc.groupby('ticker_code')['net_working_capital'].diff()
    
    # filter dataframe only in year latest year
    df_nwc = df_nwc[df_nwc['year'] == year][['ticker_code', 'net_working_capital_delta']].reset_index(drop=True)
    
    # calculate free cash flow
    df_cf = df[df['year'] == 2018][['year', 'ticker_code', 'pretax_income', 'fixed_assets', 'tax_expenses']]
    df_cf = pandas.merge(df_cf, df_nwc, how = 'inner', on = 'ticker_code').reset_index(drop = True)
    df_cf['free_cash_flow'] = df_cf['pretax_income'] \
    - df_cf['fixed_assets'] \
    - df_cf['net_working_capital_delta'] \
    - df_cf['tax_expenses']
    
    return df_cf[['year', 'ticker_code', 'free_cash_flow']].dropna().reset_index(drop=True)

In [8]:
df_cf = get_free_cash_flow(2018)